In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from models.galata import Galata
from models.kostantiniyye import Kostantiniyye
import pandas as pd
from torch.utils.data import DataLoader, random_split


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Galata(device=device)

In [ ]:
model.load_state_dict(torch.load('best_kostantiniyye_softness (1).pth'), strict= False )

In [ ]:
model.softness.load_state_dict(torch.load('best_kostantiniyye_softness (1).pth'), strict=False)

In [ ]:
model2 = Kostantiniyye(device=device)

In [ ]:
import json

In [ ]:
with open('data/json/softness_test.json', 'r') as f:
    file = json.load(f)

In [ ]:
model2.to(device)

In [ ]:
from imagebind.data import load_and_transform_vision_data

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
import json
from imagebind.data import load_and_transform_vision_data

In [ ]:
import torch

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open(f'data/json/{touch}_train.json', 'r') as f:
    data = json.load(f)

In [ ]:
# test
img = []
portion = []
label = []
for i in data:
    for im in i['img']:
        img.append(load_and_transform_vision_data([im], device=device).to('cpu')[0])
        portion.append(i['portion'])
        label.append(i['label']-1)


In [ ]:
train = TensorDataset(torch.stack(img).to('cpu'), torch.Tensor(portion).to('cpu'), torch.Tensor(label).to('cpu'))

In [ ]:
torch.save(train, f'im_{touch}_train_set.pth')

In [ ]:
touch = 'softness'

In [ ]:
with open(f'data/json/{touch}_test.json', 'r') as f:
    data = json.load(f)

In [ ]:
# test
img = []
portion = []
label = []
for i in data:
    img.append(load_and_transform_vision_data([i['img']], device=device).to('cpu')[0])
    portion.append(i['portion'])
    label.append(i['label']-1)


In [ ]:
test = TensorDataset(torch.stack(img).to('cpu'), torch.Tensor(portion).to('cpu'), torch.Tensor(label).to('cpu'))

In [ ]:
torch.save(test, f'im_{touch}_test_set.pth')

In [ ]:
img[0].shape

In [ ]:
galata = Galata(device=device)

In [ ]:
with torch.no_grad():
    model2.eval()
    for item in file:
        im = load_and_transform_vision_data([item['img']], device)
        po = torch.Tensor([item['portion']]).to(device)
        la = item['label']
        re = model2(im, po)
        print(la-1, re)

In [ ]:
with torch.no_grad():
    model.eval()
    for item in file:
        im = item['img']
        po = torch.Tensor([item['portion']]).to(device)
        la = item['label']
        re = model(im, po)
        print(la-1, re)

In [ ]:
image = [pre(Image.open(i['img'])).unsqueeze(0)for i in j]
portion = [i['portion']for i in j]
label = [i['label'] -1 for i in j]

In [ ]:
da = {}
for i in['softness', 'smoothness', 'flexibility', 'thickness']:
    for j in ['train']:
        with open(f'data/json/{i}_{j}.json')as f:
            j = json.load(f)
        j = [{'img':im, 'portion':i['portion'], 'label' :i['label']}for i in j for im in i['img']]
        image = torch.stack([pre(Image.open(i['img']))for i in j])
        portion = torch.Tensor([i['portion']for i in j])
        label = torch.Tensor([i['label'] -1 for i in j])
        print(image.shape)

        torch.save(TensorDataset(image, portion, label), f'datasets/{i}_train.pth')

In [ ]:
for i in['softness', 'smoothness', 'flexibility', 'thickness']:
    for j in ['test']:
        with open(f'data/json/{i}_{j}.json')as f:
            j = json.load(f)
        image = torch.stack([pre(Image.open(i['img']))for i in j])
        portion = torch.Tensor([i['portion']for i in j])
        label = torch.Tensor([i['label'] -1 for i in j])
        print(image.shape)
        torch.save(TensorDataset(image, portion, label), f'datasets/{i}_test.pth')

In [ ]:
[{'img':im, 'portion':i['portion'], 'label' :i['label']}for i in j for im in i['img']]

In [ ]:
torch.stack([torch.load('datasets/dataset_flexibility_train.pth')[0][0], torch.load('datasets/dataset_flexibility_train.pth')[1][0]]).shape

In [ ]:
image.shape